In [19]:
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/2"

In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

C:\Users\bekasker\miniconda3\envs\python3.10\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [58]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(preprocessor_url)
encoder_inputs = preprocessor(text_input)

In [59]:
encoder = hub.KerasLayer(encoder_url, trainable=True)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]

In [60]:
layer = tf.keras.layers.Dropout(0.1, name='dropout')(pooled_output)
layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])

In [61]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['input_3[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [45]:
from tensorflow import keras
from tensorflow.keras import callbacks, metrics 

In [62]:
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-5), 
    loss='binary_crossentropy', 
    metrics=METRICS
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-4,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

In [63]:
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=5,
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/5
513/513 [==============================] - 611s 1s/step - loss: 0.4405 - accuracy: 0.7857 - precision: 0.7759 - recall: 0.8033 - val_loss: 0.3354 - val_accuracy: 0.8567 - val_precision: 0.8297 - val_recall: 0.8976
Epoch 2/5
513/513 [==============================] - 592s 1s/step - loss: 0.3024 - accuracy: 0.8704 - precision: 0.8674 - recall: 0.8745 - val_loss: 0.2939 - val_accuracy: 0.8729 - val_precision: 0.8881 - val_recall: 0.8532
Epoch 3/5
513/513 [==============================] - 591s 1s/step - loss: 0.2523 - accuracy: 0.8970 - precision: 0.8946 - recall: 0.9000 - val_loss: 0.2726 - val_accuracy: 0.8880 - val_precision: 0.8827 - val_recall: 0.8948
Epoch 4/5
513/513 [==============================] - 599s 1s/step - loss: 0.2080 - accuracy: 0.9156 - precision: 0.9150 - recall: 0.9163 - val_loss: 0.2830 - val_accuracy: 0.8877 - val_precision: 0.9074 - val_recall: 0.8635
Epoch 5/5
513/513 [==============================] - 600s 1s/step - loss: 0.1791 - accuracy: 0.9291 - pr